<a href="https://colab.research.google.com/github/itsmihhhhhh/Luno-CryptoTradingBot/blob/master/ETHBTC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install luno_python
!pip install mpl_finance
!pip install pyti
!pip install numpy

In [12]:
from luno_python.client import Client
c = Client(api_key_id='dfqpu5an2kh7q', api_key_secret='UWjQZBJxoNEOKL1id59VaFBSBORvuAdeR2ZHA6_vdso')
try:
  ticker = c.get_ticker(pair='ETHXBT')
  print(ticker)
except Exception as e:
  print(e)

ticker['last_trade']

{'pair': 'ETHXBT', 'timestamp': 1597586913426, 'bid': '0.035838', 'ask': '0.035876', 'last_trade': '0.035838', 'rolling_24_hour_volume': '942.06', 'status': 'ACTIVE'}


'0.035838'

In [ ]:
res = c.get_order_book(pair='ETHXBT')
res

In [14]:
balance = c.get_balances(assets=['ETH','XBT'])
balXBT = balance['balance'][0]['balance']
balETH = balance['balance'][1]['balance']
balETH
balXBT
round(float(balETH),2)
balETH


'0.099352'

In [15]:
closePrices = c.list_trades(pair='ETHXBT')
closePrices['trades'][0]['price']

'0.035838'

In [16]:
enter_trade = True

In [17]:
pair = 'ETHXBT'
closePrices = c.list_trades(pair='ETHXBT')

price = ticker['last_trade']

# This function places a market order in the direction BuySell, "B" = Buy, "S" = Sell, uses symbol, amount, stop, limit
def enter(BuySell):
    global enter_trade 

    if BuySell == 'B':
      '''
      if enter_trade == True:
        print("Entered trade already, waiting to sell")
      else:
      '''
      try:
        if (price == float(res['asks'][0]['price'])):
          c.post_market_order(pair='ETHXBT', type='BUY', volume='0.01')
          print("Buy Successfully at price: ", closePrices['trades'][0]['price'])
          enter_trade = True
        else:
          c.post_limit_order(pair='ETHXBT', type='BID', volume='0.02', price = closePrices['trades'][0]['price'])
          print("Post limit order, buy at price: ", closePrices['trades'][0]['price'])
          enter_trade = True
      except Exception as e:
          print(e)
          enter_trade = False


    if BuySell == 'S':
      '''
      if enter_trade == False:
        print("Havent entered trade")
      else:
      '''
      try:
        if (price == float(res['bids'][0]['price'])):
          c.post_market_order(pair='ETHXBT', type='SELL', base_volume='0.03')
          print("Sell Successfully at price.", closePrices['trades'][0]['price'])
          enter_trade = False
        else:
          c.post_limit_order(pair='ETHXBT', type='ASK', volume='0.04', price = closePrices['trades'][0]['price'])
          print("Post limit order, sell at price.", closePrices['trades'][0]['price'])
          enter_trade = False
      except Exception as e:
          print(e)
          enter_trade = True



In [18]:
!pip install tulipy
import tulipy as ti

rsiPeriod = 14
upper_rsi = 70.0
lower_rsi = 30.0

In [19]:
res = c.get_order_book(pair='ETHXBT')
len(res['asks'])

100

In [ ]:
import sched, time
import numpy as np


#Initiate our scheduler so we can keep checking every minute for new price changes
s = sched.scheduler(time.time, time.sleep)

# This function is run every 5 minutes

def run(sc): 

    #Get prices from Luno
    res = c.get_order_book(pair='ETHXBT')
    closePrices = c.list_trades(pair='ETHXBT')
    pricedata = []
    for x in range(0,len(closePrices['trades'])):
        pricedata.append(float(closePrices['trades'][x]['price']))
    DATA = np.array(pricedata)

    askdata = []
    for x in range(0,len(res['asks'])):
        askdata.append(float(res['asks'][x]['price']))
    DATAA = np.array(askdata)

    biddata = []
    for x in range(0,len(res['bids'])):
        biddata.append(float(res['bids'][x]['price']))
    DATAB = np.array(biddata)

    #Calculate Indicators
    #iRSI = ti.rsi(DATA, period=14)
    BB = ti.bbands(DATA, period=20, stddev=1)
    MA = ti.sma(DATA, period=5)
    MAA = ti.sma(DATAA, period=3)
    MAB = ti.sma(DATAB, period=5)

    
    if (MAB[0] < pricedata[0] < MA[0] and float(balETH)<0.08):
        print("Buying, price is low!")
        try:
          '''
          if (pricedata[0] == float(res['asks'][0]['price'])):
            c.post_market_order(pair='ETHXBT', type='BUY', volume='0.01')
            print("Buy Successfully at price: ", pricedata[0])
            enter_trade = True
          else:
          '''
          c.post_limit_order(pair='ETHXBT', type='BID', volume='0.03', price = pricedata[0])
          print("Post limit order, buy at price: ", pricedata[0])
          enter_trade = True
        except Exception as e:
          print(e)
          enter_trade = False

    
    if (MA[0] < pricedata[0] < MAA[0]):
        print("Selling, price is high!")
        try:
          '''
          if (pricedata[0] == float(res['bids'][0]['price'])):
            c.post_market_order(pair='ETHXBT', type='SELL', base_volume='0.03')
            print("Sell Successfully at price.", pricedata[0])
            enter_trade = False
          else:
          '''
          c.post_limit_order(pair='ETHXBT', type='ASK', volume='0.03', price = pricedata[0])
          print("Post limit order, sell at price.", pricedata[0])
          enter_trade = False
        except Exception as e:
          print(e)
          enter_trade = True
    
    print("Current price: ", pricedata[0])  
    '''
    print("MA bid(low): ", MAB[0])
    print("MA middle: ", MA[0])
    print("MA ask(high): ", MAA[0])
    print("BBand lower: ", BB[0][0])
    print("BBand middle: ", BB[1][0])
    print("BBand upper ", BB[2][0])
    '''
  
    print("                               Bot working, checking...")

    #call this method again every 5 minutes for new price changes
    s.enter(1000, 1, run, (sc,))
    
s.enter(1, 1, run, (s,))

s.run()